# 03 – SLA & Breach & Feature Labeling

ในโน้ตบุ๊กนี้ เราจะ:

1. เลือกใช้เฉพาะเคสที่ "เสร็จสิ้น" เพื่อคำนวณเวลาแก้ไขจริง (duration_days)
2. ใช้ข้อมูลจริงของแต่ละ `std_type` (หมวดปัญหา) มาคำนวณ SLA แบบ data-driven ด้วย quantile
3. สร้าง label `breach` (1 = เกิน SLA, 0 = ไม่เกิน SLA) สำหรับเคสที่เสร็จแล้ว → ใช้เป็น training data สำหรับ ML
4. แยกชุดข้อมูล:
   - `df_completed_for_model` → ใช้ train model
   - `df_pending_for_predict` → เอาไว้ให้ model ทำนายในอนาคต
5. เซฟไฟล์ processed + sample CSV ไว้ใช้ในขั้นตอนถัดไป


In [59]:
import pandas as pd

In [60]:
df_clean = pd.read_csv('../data/traffy_clean_150k.csv')
df_clean.head(1)

,ticket_id,std_type,type,type_list,organization,comment,comment_length,photo,photo_after,lng,...,subdistrict,district,province,province_clean,timestamp,last_activity,state,star,count_reopen,year
0,2024-CVGNCL,road,{ถนน},['ถนน'],"เขตราชเทวี,ฝ่ายเทศกิจ เขตราชเทวี",ปัญหา: บริเวณหน้าบ้านเลขที่ 425/1-2 พบคนเรร่อน...,351,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,100.54204,...,ถนนพญาไท,ราชเทวี,กรุงเทพมหานคร,กรุงเทพมหานคร,2024-07-31 02:56:49.753428+00:00,2024-11-03 18:26:10.631498+00:00,เสร็จสิ้น,5.0,0,2024


## สร้าง Feature พื้นฐานสำหรับ model

In [61]:
df_clean["timestamp"] = pd.to_datetime(df_clean["timestamp"], format="mixed", errors="coerce")

df_clean["day_of_week"] = df_clean["timestamp"].dt.dayofweek  # 0=Monday
df_clean["hour_of_day"] = df_clean["timestamp"].dt.hour
df_clean["month"] = df_clean["timestamp"].dt.month
df_clean[["comment_length", "day_of_week", "hour_of_day", "month"]].head()

,comment_length,day_of_week,hour_of_day,month
0,351,2,2,7
1,10,3,8,11
2,424,5,18,12
3,54,6,7,5
4,50,3,9,1


## สร้างระดับความเร่งด่วน (urgency) ตามหมวดปัญหา

แม้ SLA จะถูกกำหนดจากข้อมูลจริง (quantile ต่อ `std_type`)  
แต่เรายังอยากแบ่งหมวดให้เข้าใจง่าย เช่น:

- **High urgency**: เสี่ยงอันตราย/กระทบความปลอดภัยสูง เช่น ความปลอดภัย, แสงสว่าง
- **Mid urgency**: โครงสร้างพื้นฐานที่ควรแก้ในไม่กี่วัน เช่น คลอง, เสียงรบกวน
- **Low urgency**: ปัญหาที่กระทบคุณภาพชีวิตระยะกลาง เช่น สัตว์จรจัด,อื่นๆ


In [62]:
urgency_map = {
    "road": "high",
    "lighting": "high",
    "safety": "high",
    "flood": "high",

    "canal": "mid",
    "noise": "mid",
    "cleaning": "mid",

    "animal": "low",
    "other": "low",
    "unknown": "low",
}

df_pro = df_clean.copy()
df_pro["urgency"] = df_clean["std_type"].map(urgency_map).fillna('low')
df_pro["urgency"].value_counts()


urgency
high    109759
mid      21572
low      18669
Name: count, dtype: int64

## เลือกเฉพาะเคสที่ "เสร็จสิ้น" เพื่อใช้สร้าง SLA

การสร้าง SLA ต้องอิง "เวลาจริงที่แก้เสร็จแล้ว"  
ดังนั้นเราจะใช้เฉพาะ `state == "เสร็จสิ้น"`  
เพื่อคำนวณ `duration_days = last_activity - timestamp`


In [63]:

df_completed = df_pro[df_pro["state"] == "เสร็จสิ้น"].copy()

df_completed["timestamp"] = pd.to_datetime(df_completed["timestamp"], format="mixed", errors="coerce")
df_completed["last_activity"] = pd.to_datetime(df_completed["last_activity"], format="mixed", errors="coerce")

df_completed["duration_days"] = (
    df_completed["last_activity"] - df_completed["timestamp"]
).dt.total_seconds() / 86400

df_completed["duration_days"].describe(percentiles=[0.5, 0.75, 0.9, 0.95])

count    111210.000000
mean         27.001394
std          55.113170
min           0.000536
50%           4.823357
75%          20.908063
90%          84.083224
95%         152.289703
max         378.228303
Name: duration_days, dtype: float64

## สร้าง SLA ด้วย quantile ต่อ `std_type`

แนวคิด:

- Group by ตาม `std_type`
- ดู distribution ของ `duration_days`
- ใช้ **`70th` percentile (quantile 0.7)** เป็น SLA ของ `high`  
- ใช้ **`80th` percentile (quantile 0.8)** เป็น SLA ของ `mid`  
- ใช้ **`90th`percentile (quantile 0.9)** เป็น SLA ของ `low`  
แบ่งตามระดับความสำคัญของปัญหาตามที่ระบุไว้


In [64]:
quantile_by_urgency = {
    "high": 0.7,
    "mid": 0.8,
    "low": 0.9,
}

default_q = 0.8


def compute_sla_for_type(group):

    urgency = group["urgency"].mode().iloc[0] 

    # เลือก quantile ตาม urgency
    q = quantile_by_urgency.get(urgency, default_q)

    return group["duration_days"].quantile(q)

sla_series = (
    df_completed
    .groupby("std_type")
    .apply(compute_sla_for_type)
)

sla_series.name = "SLA_days"
sla_series
sla_map = sla_series.to_dict()
sla_map


/var/folders/83/f60f0cr13t30wlv9fk68hxdm0000gn/T/ipykernel_81002/3779950765.py:22: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(compute_sla_for_type)


{'animal': 72.86693309369213,
 'canal': 32.95179462729631,
 'cleaning': 11.866649166648148,
 'flood': 11.670945969684022,
 'lighting': 73.68319893052893,
 'noise': 28.791555815460676,
 'other': 57.633311016612474,
 'road': 12.46195016349537,
 'safety': 14.08306067712731,
 'unknown': 90.06795582402778}

## ผูก SLA กลับไปที่ข้อมูล

- ทุกเคสที่มี `std_type` เราจะรู้ว่า SLA ของมันคือกี่วัน (`SLA_days`)
- ใน training (เฉพาะเสร็จสิ้น) เราจะใช้ `duration_days` เทียบกับ `SLA_days` เพื่อสร้าง label `breach`
- `breach = 1` ถ้า `duration_days > SLA_days`
- `breach = 0` ถ้า `duration_days <= SLA_days`

ใช้เฉพาะ `df_completed` เท่านั้นเป็น training data  
ส่วน `state อื่น ๆ` (กำลังดำเนินการ/รอรับเรื่อง) → ไว้ให้ model ทำนายในภายหลัง

In [65]:
# ใส่ SLA ให้ทุกแถว (ทั้ง completed + pending)
df_pro["SLA_days"] = df_pro["std_type"].map(sla_map)

df_completed["SLA_days"] = df_completed["std_type"].map(sla_map)
df_completed[["std_type", "duration_days", "SLA_days"]].head()

,std_type,duration_days,SLA_days
0,road,95.645381,12.461950
1,cleaning,1.730293,11.866649
3,road,36.955478,12.461950
4,road,12.944869,12.461950
5,road,1.997139,12.461950


In [66]:
df_completed["breach"] = (
    df_completed["duration_days"] > df_completed["SLA_days"]
).astype(int)

df_completed["breach"].value_counts(normalize=True)
df_completed


,ticket_id,std_type,type,type_list,organization,comment,comment_length,photo,photo_after,lng,...,star,count_reopen,year,day_of_week,hour_of_day,month,urgency,duration_days,SLA_days,breach
0,2024-CVGNCL,road,{ถนน},['ถนน'],"เขตราชเทวี,ฝ่ายเทศกิจ เขตราชเทวี",ปัญหา: บริเวณหน้าบ้านเลขที่ 425/1-2 พบคนเรร่อน...,351,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,100.54204,...,5.0,0,2024,2,2,7,high,95.645381,12.461950,1
1,2024-ATFNH8,cleaning,{ความสะอาด},['ความสะอาด'],"เขตลาดกระบัง,ฝ่ายรักษาความสะอาดฯ เขตลาดกระบัง",ขยะไม่เก็บ,10,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,100.73608,...,4.0,0,2024,3,8,11,mid,1.730293,11.866649,0
3,2024-ET2E9B,road,"{ถนน,แสงสว่าง}","['ถนน', 'แสงสว่าง']","เขตคลองสามวา,สำนักการโยธา กทม.,การไฟฟ้านครหลวง...",ไฟส่องสว่าง บริเวณเกาะกลางดับ ถนนสามวา (สามวา ...,54,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,100.72685,...,NaN,0,2024,6,7,5,high,36.955478,12.461950,1
4,2024-DRE2UU,road,{ถนน},['ถนน'],"เขตบางพลัด,สำนักการโยธา กทม.,สำนักงานก่อสร้างแ...",ซ่อมถนนตรงนี้ไหนครับเป็นร่องตรงกลางยาว 10 เมตร...,50,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,100.48622,...,NaN,0,2024,3,9,1,high,12.944869,12.461950,1
5,2024-FA4DGQ,road,"{ทางเท้า,คนจรจัด}","['ทางเท้า', 'คนจรจัด']","เขตวัฒนา,เขตคลองเตย,ฝ่ายเทศกิจ เขตวัฒนา",ที่บริเวณทางลงจากทางเดินไป BTS พร้อมพงศ์ ด้าน ...,262,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,100.56846,...,NaN,0,2024,2,5,7,high,1.997139,12.461950,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149995,2024-BECXPX,road,{ป้าย},['ป้าย'],"เขตตลิ่งชัน,ฝ่ายเทศกิจ เขตตลิ่งชัน",ป้ายโฆษณาติดเต็มหมด,19,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,100.44558,...,5.0,0,2024,0,9,1,high,5.238576,12.461950,0
149996,2024-F4KZ3N,road,{ทางเท้า},['ทางเท้า'],"เขตสาทร,ฝ่ายเทศกิจ เขตสาทร",ศูนย์การค้าวรรัตน์\n1.เจ้าหน้าที่อ้างในการตอบเ...,658,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,100.53110,...,NaN,0,2024,0,12,10,high,64.671013,12.461950,1
149997,2024-FTTYU8,unknown,{},['unknown'],"เขตวัฒนา,ฝ่ายสิ่งแวดล้อมฯ เขตวัฒนา",มัสยิดอาซานใส่ไมค์ดังตกใจตื่นอีกแล้ว ทุกวัน จะ...,60,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,100.59678,...,NaN,0,2024,1,22,3,low,2.098330,90.067956,0
149998,2024-FRRHX8,safety,{PM2.5},['PM2.5'],"เขตธนบุรี,สำนักการโยธา กทม.,สำนักสิ่งแวดล้อม ก...",อยากให้สนง.เขต และสนย. กำชับผู้รับเหมา ช่วยเอา...,74,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,100.47827,...,1.0,0,2024,5,8,2,high,43.807047,14.083061,1


In [67]:
breach_by_type = (
    df_completed
    .groupby("std_type")["breach"]
    .mean()
    .sort_values(ascending=False)
)
breach_by_type


std_type
flood       0.300087
lighting    0.300075
safety      0.300000
road        0.299994
canal       0.200291
cleaning    0.200049
noise       0.200026
other       0.102484
unknown     0.099991
animal      0.099945
Name: breach, dtype: float64

In [69]:
feature_cols = [
    "ticket_id",
    "std_type",
    "district",
    "subdistrict",
    "province_clean",
    "urgency",
    "comment_length",
    "day_of_week",
    "hour_of_day",
    "month",
    "lng",
    "lat",
    "SLA_days",
]

# 1) ชุดสำหรับ train model (เฉพาะเคสที่เสร็จสิ้น + มี breach label)
df_completed_for_model = df_completed[feature_cols + ["breach"]].copy()

# 2) ชุดสำหรับให้ model ทำนาย (ยังไม่เสร็จสิ้น)
df_pending_for_predict = (
    df_pro[df_pro["state"] != "เสร็จสิ้น"][feature_cols].copy()
)

print("df_pro (all states):", df_pro.shape)
print("df_completed_for_model (train):", df_completed_for_model.shape)
print("df_pending_for_predict (to predict):", df_pending_for_predict.shape)


df_pro (all states): (150000, 26)
df_completed_for_model (train): (111210, 14)
df_pending_for_predict (to predict): (38790, 13)


In [70]:
df_pro.to_csv("../data/traffy_sla_labeled_full.csv", index=False)

df_completed_for_model.to_csv("../data/traffy_completed_for_model.csv", index=False)

df_pending_for_predict.to_csv("../data/traffy_pending_for_predict.csv", index=False)


In [71]:
n_sample = 100  

df_completed_for_model.sample(
    n_sample,
    random_state=42
).to_csv("../data_samples/02-completed_for_model_sample.csv", index=False)

df_pending_for_predict.sample(
    n_sample,
    random_state=42
).to_csv("../data_samples/02-pending_for_predict_sample.csv", index=False)


In [72]:
df_pro.head()

,ticket_id,std_type,type,type_list,organization,comment,comment_length,photo,photo_after,lng,...,last_activity,state,star,count_reopen,year,day_of_week,hour_of_day,month,urgency,SLA_days
0,2024-CVGNCL,road,{ถนน},['ถนน'],"เขตราชเทวี,ฝ่ายเทศกิจ เขตราชเทวี",ปัญหา: บริเวณหน้าบ้านเลขที่ 425/1-2 พบคนเรร่อน...,351,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,100.54204,...,2024-11-03 18:26:10.631498+00:00,เสร็จสิ้น,5.0,0,2024,2,2,7,high,12.461950
1,2024-ATFNH8,cleaning,{ความสะอาด},['ความสะอาด'],"เขตลาดกระบัง,ฝ่ายรักษาความสะอาดฯ เขตลาดกระบัง",ขยะไม่เก็บ,10,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,100.73608,...,2024-11-30 01:58:13.419161+00:00,เสร็จสิ้น,4.0,0,2024,3,8,11,mid,11.866649
2,ZN3ZZV,road,"{ถนน,กีดขวาง}","['ถนน', 'กีดขวาง']","เขตลาดพร้าว,ฝ่ายเทศกิจ เขตลาดพร้าว",ปัญหา: ริมถนนดังกล่าว บริเวณหน้าเซ่เวน อีเลฟเว...,424,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,100.59014,...,2024-12-25 16:53:44.505932+00:00,กำลังดำเนินการ,NaN,3,2024,5,18,12,high,12.461950
3,2024-ET2E9B,road,"{ถนน,แสงสว่าง}","['ถนน', 'แสงสว่าง']","เขตคลองสามวา,สำนักการโยธา กทม.,การไฟฟ้านครหลวง...",ไฟส่องสว่าง บริเวณเกาะกลางดับ ถนนสามวา (สามวา ...,54,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,100.72685,...,2024-06-18 06:47:53.894049+00:00,เสร็จสิ้น,NaN,0,2024,6,7,5,high,12.461950
4,2024-DRE2UU,road,{ถนน},['ถนน'],"เขตบางพลัด,สำนักการโยธา กทม.,สำนักงานก่อสร้างแ...",ซ่อมถนนตรงนี้ไหนครับเป็นร่องตรงกลางยาว 10 เมตร...,50,https://storage.googleapis.com/traffy_public_b...,https://storage.googleapis.com/traffy_public_b...,100.48622,...,2024-01-24 08:13:39.004389+00:00,เสร็จสิ้น,NaN,0,2024,3,9,1,high,12.461950


In [73]:
df_completed_for_model.head()

,ticket_id,std_type,district,subdistrict,province_clean,urgency,comment_length,day_of_week,hour_of_day,month,lng,lat,SLA_days,breach
0,2024-CVGNCL,road,ราชเทวี,ถนนพญาไท,กรุงเทพมหานคร,high,351,2,2,7,100.54204,13.75420,12.461950,1
1,2024-ATFNH8,cleaning,ลาดกระบัง,คลองสองต้นนุ่น,กรุงเทพมหานคร,mid,10,3,8,11,100.73608,13.73114,11.866649,0
3,2024-ET2E9B,road,คลองสามวา,บางชัน,กรุงเทพมหานคร,high,54,6,7,5,100.72685,13.84308,12.461950,1
4,2024-DRE2UU,road,บางพลัด,บางยี่ขัน,กรุงเทพมหานคร,high,50,3,9,1,100.48622,13.77831,12.461950,1
5,2024-FA4DGQ,road,คลองเตย,คลองตัน,กรุงเทพมหานคร,high,262,2,5,7,100.56846,13.73143,12.461950,0


In [74]:
df_pending_for_predict.head()

,ticket_id,std_type,district,subdistrict,province_clean,urgency,comment_length,day_of_week,hour_of_day,month,lng,lat,SLA_days
2,ZN3ZZV,road,ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,high,424,5,18,12,100.59014,13.80482,12.461950
6,2024-N9H4PR,road,พญาไท,สามเสนใน,กรุงเทพมหานคร,high,723,5,11,3,100.54818,13.76384,12.461950
9,2024-GDN42R,road,หนองจอก,คลองสิบสอง,กรุงเทพมหานคร,high,98,5,3,12,100.86285,13.89961,12.461950
10,2024-LNHRXD,noise,ปทุมวัน,รองเมือง,กรุงเทพมหานคร,mid,117,0,14,12,100.52070,13.74034,28.791556
11,2024-LMCD6M,unknown,วัฒนา,พระโขนงเหนือ,กรุงเทพมหานคร,low,12,1,7,6,100.59589,13.72575,90.067956
